- session intialization

In [1]:
from pyspark.sql import SparkSession;
from pyspark.context import SparkContext
spark = SparkSession \
    .builder \
    .appName("Dtree_classification") \
    .getOrCreate()

sc = spark.sparkContext

In [2]:
spark

In [3]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder, StringIndexer

In [18]:
df = spark.read.csv('./data/airbnb.csv',header=True, inferSchema=True)
df.show(5)

+-----------------+----------------------+----------------------+-----------+------------+-------------+---------------+------------+---------+--------+----+--------+-------------------+---------------+----------------------+--------------+-----------------+---------------------------------+--------------------+-------------------+-----+-------------+--------------+
|host_is_superhost|host_identity_verified|neighbourhood_cleansed|   latitude|   longitude|property_type|      room_type|accommodates|bathrooms|bedrooms|beds|bed_type|Number of amenities|guests_included|price_per_extra_person|minimum_nights|number_of_reviews|number_days_btw_first_last_review|review_scores_rating|cancellation_policy|price|price_gte_150|price_category|
+-----------------+----------------------+----------------------+-----------+------------+-------------+---------------+------------+---------+--------+----+--------+-------------------+---------------+----------------------+--------------+-----------------+----

In [19]:
df.describe().show()

+-------+-------------------+----------------------+----------------------+--------------------+-------------------+-------------+---------------+------------------+------------------+------------------+------------------+--------+-------------------+------------------+----------------------+------------------+------------------+---------------------------------+--------------------+-------------------+------------------+------------------+--------------+
|summary|  host_is_superhost|host_identity_verified|neighbourhood_cleansed|            latitude|          longitude|property_type|      room_type|      accommodates|         bathrooms|          bedrooms|              beds|bed_type|Number of amenities|   guests_included|price_per_extra_person|    minimum_nights| number_of_reviews|number_days_btw_first_last_review|review_scores_rating|cancellation_policy|             price|     price_gte_150|price_category|
+-------+-------------------+----------------------+----------------------+-----

In [20]:
from pyspark.sql.functions import col, sum as spark_sum

missing_counts = df.select([spark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

# Show the result
missing_counts.show()

+-----------------+----------------------+----------------------+--------+---------+-------------+---------+------------+---------+--------+----+--------+-------------------+---------------+----------------------+--------------+-----------------+---------------------------------+--------------------+-------------------+-----+-------------+--------------+
|host_is_superhost|host_identity_verified|neighbourhood_cleansed|latitude|longitude|property_type|room_type|accommodates|bathrooms|bedrooms|beds|bed_type|Number of amenities|guests_included|price_per_extra_person|minimum_nights|number_of_reviews|number_days_btw_first_last_review|review_scores_rating|cancellation_policy|price|price_gte_150|price_category|
+-----------------+----------------------+----------------------+--------+---------+-------------+---------+------------+---------+--------+----+--------+-------------------+---------------+----------------------+--------------+-----------------+---------------------------------+------

In [21]:
df = df.dropna(how='any')

In [22]:
missing_counts = df.select([spark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

# Show the result
missing_counts.show()

+-----------------+----------------------+----------------------+--------+---------+-------------+---------+------------+---------+--------+----+--------+-------------------+---------------+----------------------+--------------+-----------------+---------------------------------+--------------------+-------------------+-----+-------------+--------------+
|host_is_superhost|host_identity_verified|neighbourhood_cleansed|latitude|longitude|property_type|room_type|accommodates|bathrooms|bedrooms|beds|bed_type|Number of amenities|guests_included|price_per_extra_person|minimum_nights|number_of_reviews|number_days_btw_first_last_review|review_scores_rating|cancellation_policy|price|price_gte_150|price_category|
+-----------------+----------------------+----------------------+--------+---------+-------------+---------+------------+---------+--------+----+--------+-------------------+---------------+----------------------+--------------+-----------------+---------------------------------+------

In [23]:
df.show(1)

+-----------------+----------------------+----------------------+-----------+------------+-------------+------------+------------+---------+--------+----+--------+-------------------+---------------+----------------------+--------------+-----------------+---------------------------------+--------------------+-------------------+-----+-------------+--------------+
|host_is_superhost|host_identity_verified|neighbourhood_cleansed|   latitude|   longitude|property_type|   room_type|accommodates|bathrooms|bedrooms|beds|bed_type|Number of amenities|guests_included|price_per_extra_person|minimum_nights|number_of_reviews|number_days_btw_first_last_review|review_scores_rating|cancellation_policy|price|price_gte_150|price_category|
+-----------------+----------------------+----------------------+-----------+------------+-------------+------------+------------+---------+--------+----+--------+-------------------+---------------+----------------------+--------------+-----------------+-------------

In [24]:
df.printSchema()

root
 |-- host_is_superhost: integer (nullable = true)
 |-- host_identity_verified: integer (nullable = true)
 |-- neighbourhood_cleansed: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- property_type: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- accommodates: integer (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- beds: integer (nullable = true)
 |-- bed_type: string (nullable = true)
 |-- Number of amenities: integer (nullable = true)
 |-- guests_included: integer (nullable = true)
 |-- price_per_extra_person: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- number_days_btw_first_last_review: integer (nullable = true)
 |-- review_scores_rating: integer (nullable = true)
 |-- cancellation_policy: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- 

In [25]:
df.dtypes

[('host_is_superhost', 'int'),
 ('host_identity_verified', 'int'),
 ('neighbourhood_cleansed', 'string'),
 ('latitude', 'double'),
 ('longitude', 'double'),
 ('property_type', 'string'),
 ('room_type', 'string'),
 ('accommodates', 'int'),
 ('bathrooms', 'double'),
 ('bedrooms', 'int'),
 ('beds', 'int'),
 ('bed_type', 'string'),
 ('Number of amenities', 'int'),
 ('guests_included', 'int'),
 ('price_per_extra_person', 'int'),
 ('minimum_nights', 'int'),
 ('number_of_reviews', 'int'),
 ('number_days_btw_first_last_review', 'int'),
 ('review_scores_rating', 'int'),
 ('cancellation_policy', 'string'),
 ('price', 'int'),
 ('price_gte_150', 'int'),
 ('price_category', 'string')]

In [30]:
from pyspark.sql.types import StringType

def is_categorical(column):
    return isinstance(df.schema[column].dataType, StringType)

# Select only categorical columns
categorical_columns = [column for column in df.columns if is_categorical(column)]

In [31]:
categorical_columns

['neighbourhood_cleansed',
 'property_type',
 'room_type',
 'bed_type',
 'cancellation_policy',
 'price_category']

- Some algorithms, like tree-based algorithms (e.g., decision trees, random forests), can naturally handle categorical features without explicit encoding. In such cases, using the combined feature vector may be more convenient.

- Before building the model, we need to assemble the input features into a single feature vector using the VectorAssembler class. Then, we will split the dataset into a training set (80%) and a testing set (20%).

- Indexing

In [34]:
indexed_df = df
for col in categorical_columns:
    indexer = StringIndexer(inputCol=col, outputCol=col+'_indexed')
    indexed_df = indexer.fit(indexed_df).transform(indexed_df)
    indexed_df = indexed_df.drop(col)
indexed_df.show()

+-----------------+----------------------+-----------+------------+------------+---------+--------+----+-------------------+---------------+----------------------+--------------+-----------------+---------------------------------+--------------------+-----+-------------+------------------------------+---------------------+-----------------+----------------+---------------------------+----------------------+
|host_is_superhost|host_identity_verified|   latitude|   longitude|accommodates|bathrooms|bedrooms|beds|Number of amenities|guests_included|price_per_extra_person|minimum_nights|number_of_reviews|number_days_btw_first_last_review|review_scores_rating|price|price_gte_150|neighbourhood_cleansed_indexed|property_type_indexed|room_type_indexed|bed_type_indexed|cancellation_policy_indexed|price_category_indexed|
+-----------------+----------------------+-----------+------------+------------+---------+--------+----+-------------------+---------------+----------------------+--------------+

In [36]:
df = indexed_df
df.dtypes

[('host_is_superhost', 'int'),
 ('host_identity_verified', 'int'),
 ('latitude', 'double'),
 ('longitude', 'double'),
 ('accommodates', 'int'),
 ('bathrooms', 'double'),
 ('bedrooms', 'int'),
 ('beds', 'int'),
 ('Number of amenities', 'int'),
 ('guests_included', 'int'),
 ('price_per_extra_person', 'int'),
 ('minimum_nights', 'int'),
 ('number_of_reviews', 'int'),
 ('number_days_btw_first_last_review', 'int'),
 ('review_scores_rating', 'int'),
 ('price', 'int'),
 ('price_gte_150', 'int'),
 ('neighbourhood_cleansed_indexed', 'double'),
 ('property_type_indexed', 'double'),
 ('room_type_indexed', 'double'),
 ('bed_type_indexed', 'double'),
 ('cancellation_policy_indexed', 'double'),
 ('price_category_indexed', 'double')]

In [37]:
X = df.drop('price')
Y = df.select('price')

In [38]:
Y.show(5)

+-----+
|price|
+-----+
|   65|
|   65|
|   75|
|   79|
|   75|
+-----+
only showing top 5 rows



In [39]:
X.show(5)

+-----------------+----------------------+-----------+------------+------------+---------+--------+----+-------------------+---------------+----------------------+--------------+-----------------+---------------------------------+--------------------+-------------+------------------------------+---------------------+-----------------+----------------+---------------------------+----------------------+
|host_is_superhost|host_identity_verified|   latitude|   longitude|accommodates|bathrooms|bedrooms|beds|Number of amenities|guests_included|price_per_extra_person|minimum_nights|number_of_reviews|number_days_btw_first_last_review|review_scores_rating|price_gte_150|neighbourhood_cleansed_indexed|property_type_indexed|room_type_indexed|bed_type_indexed|cancellation_policy_indexed|price_category_indexed|
+-----------------+----------------------+-----------+------------+------------+---------+--------+----+-------------------+---------------+----------------------+--------------+------------

In [40]:
assembler = VectorAssembler(inputCols=X.columns, outputCol="features")
df = assembler.transform(df)

- splitting Data

In [41]:
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

In [42]:
train_data.show(5)

+-----------------+----------------------+-----------+------------+------------+---------+--------+----+-------------------+---------------+----------------------+--------------+-----------------+---------------------------------+--------------------+-----+-------------+------------------------------+---------------------+-----------------+----------------+---------------------------+----------------------+--------------------+
|host_is_superhost|host_identity_verified|   latitude|   longitude|accommodates|bathrooms|bedrooms|beds|Number of amenities|guests_included|price_per_extra_person|minimum_nights|number_of_reviews|number_days_btw_first_last_review|review_scores_rating|price|price_gte_150|neighbourhood_cleansed_indexed|property_type_indexed|room_type_indexed|bed_type_indexed|cancellation_policy_indexed|price_category_indexed|            features|
+-----------------+----------------------+-----------+------------+------------+---------+--------+----+-------------------+------------

In [46]:
train_data.select('price').describe().show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|              2229|
|   mean| 164.3472409152086|
| stddev|102.33563086478506|
|    min|                11|
|    max|               650|
+-------+------------------+



In [49]:
from pyspark.ml.regression import DecisionTreeRegressor

In [50]:
dt_classifier = DecisionTreeRegressor(labelCol='price', featuresCol="features")

model = dt_classifier.fit(train_data)

In [51]:
predictions = model.transform(test_data)

In [53]:
evaluator = MulticlassClassificationEvaluator(labelCol="price", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(f"Test Accuracy: {accuracy:.2f}")

Test Accuracy: 0.07


In [54]:
print(model.toDebugString)

DecisionTreeRegressionModel: uid=DecisionTreeRegressor_ac1a701a3f46, depth=5, numNodes=57, numFeatures=22
  If (feature 21 in {0.0,1.0,3.0})
   If (feature 15 <= 0.5)
    If (feature 21 in {3.0})
     If (feature 4 <= 1.5)
      If (feature 11 <= 7.5)
       Predict: 54.73809523809524
      Else (feature 11 > 7.5)
       Predict: 40.666666666666664
     Else (feature 4 > 1.5)
      If (feature 13 <= 451.5)
       Predict: 61.506726457399104
      Else (feature 13 > 451.5)
       Predict: 68.32183908045977
    Else (feature 21 not in {3.0})
     If (feature 18 in {1.0,2.0})
      If (feature 16 in {0.0,3.0,4.0,5.0,8.0,9.0,12.0,13.0,16.0,17.0,18.0,20.0,21.0,24.0})
       Predict: 93.60507246376811
      Else (feature 16 not in {0.0,3.0,4.0,5.0,8.0,9.0,12.0,13.0,16.0,17.0,18.0,20.0,21.0,24.0})
       Predict: 111.95762711864407
     Else (feature 18 not in {1.0,2.0})
      If (feature 16 in {0.0,3.0,4.0,5.0,8.0,9.0,12.0,13.0,16.0,18.0,20.0})
       Predict: 117.36904761904762
      Else (